---

# Process the Performance Report

---

In [1]:
# package imports go here
import pandas as pd
import numpy as np
import fastparquet as fp
import os
import sys
import pickle
import matplotlib.pyplot as plt
import importlib
import config
import time

sys.path.insert(1, config.package_path)
import ml_analysis as mlanlys

In [2]:
start_time = time.time()

In [3]:
# Path to performance report
report_path = 'reports/'
performance_report              = report_path + 'performance_report.pkl'
performance_report_text_file    = report_path + 'performance_report.txt'

---

## 1. Read performance report

---

In [4]:
if os.path.exists(performance_report):
    # Load Performance Report
    with open(performance_report, 'rb') as file: perf_report = pickle.load(file)
    print(f"The file {performance_report} exists and the Performance Report was read successfully")
else:
    print(f"******************************************************")
    print(f"The file: '{performance_report}' DOES NOT EXIST")
    print(f"******************************************************")

The file reports/performance_report.pkl exists and the Performance Report was read successfully


---

## 2. Write the Performance Report

---

In [5]:

perf_datasets_in = perf_report.keys()

perf_datasets_in

dict_keys(['Base Dataset', '2.0 StandardScaler Dataset', '1 Base Dataset', '2.1 MinMaxScaler Dataset', '3 Binary Dataset', '4 RandomUnderSampler Dataset', '5 RandomOverSampler Dataset', '6 ClusterCentroids Dataset', '7 SMOTE Dataset', '8 SMOTEENN Dataset'])

#### 2.1 Modify the order of the reported datasets

---

In [6]:
# Use this cell to re-order the datasets for the report

perf_datasets = perf_datasets_in

#### 2.2 Print the report

---

In [7]:
def print_performance_report(perf_report):
    # Print header and then every performance report in the reports dictionary

    print(f"**********************************************************")
    print(f"Diabetes Predictions Performance Summary Report")
    print(f"**********************************************************")

    print(f"----------------------------------------")
    print(f"Report Summary")
    print(f"----------------------------------------")

    for report in perf_report:
        print(f"Performance for: {report}")

    print(f"----------------------------------------")

    for report in perf_report:
        report_dict = perf_report[report]
        dataset_size = report_dict['dataset_size']
        report_df = report_dict['report']
        print(f"----------------------------------------")
        print(f"Performance for: {report}")
        print(f"Dataset Size:    {dataset_size[0]} Rows, {dataset_size[1]} Columns")
        print(f"----------------------------------------")
        print(f"{report_df.to_string(index=False)}")

In [8]:
# Generate the Performance Report and send prints to osc.stdout
with mlanlys.OutStreamCapture() as osc:
    print_performance_report(perf_report)

In [9]:
# osc.stdout contains the details of the performance report
# write the performance report to the performance_report_text_file
with open(performance_report_text_file, "w") as file:
    file.write(osc.stdout)

In [10]:
# Display the performance report here:
print(osc.stdout)

**********************************************************
Diabetes Predictions Performance Summary Report
**********************************************************
----------------------------------------
Report Summary
----------------------------------------
Performance for: Base Dataset
Performance for: 2.0 StandardScaler Dataset
Performance for: 1 Base Dataset
Performance for: 2.1 MinMaxScaler Dataset
Performance for: 3 Binary Dataset
Performance for: 4 RandomUnderSampler Dataset
Performance for: 5 RandomOverSampler Dataset
Performance for: 6 ClusterCentroids Dataset
Performance for: 7 SMOTE Dataset
Performance for: 8 SMOTEENN Dataset
----------------------------------------
----------------------------------------
Performance for: Base Dataset
Dataset Size:    247076 Rows, 37 Columns
----------------------------------------
                     model slice    score  balanced_accuracy  roc_auc_score
      KNeighborsClassifier Train 0.874673           0.460255       0.936634
     

In [11]:
print(f"Completed: Execution Time %s seconds:" % round((time.time() - start_time),2) )

Completed: Execution Time 0.13 seconds:


## 3. Merge Performance Metrics into a single dataframe

In [12]:
# Concatenate all the Reports into one dataframe
df = pd.DataFrame()

for report in perf_report:
    report_dict = perf_report[report]
    report_df = report_dict['report']
    df = pd.concat([df, report_df], ignore_index=True)

In [13]:
df.rename(columns={ 'new_column': 'dataset'}, inplace=True)
df

,model,slice,score,balanced_accuracy,roc_auc_score,dataset,Mean Squared Error,Accuracy,Precision,Recall,F1-score,Specificity,False Positive Rate,Matthews Correlation Coefficient
0,KNeighborsClassifier,Train,0.874673,0.460255,0.936634,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,KNeighborsClassifier,Test,0.814243,0.356203,0.559228,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,DecisionTreeClassifier,Train,1.000000,1.000000,1.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,DecisionTreeClassifier,Test,0.763020,0.403209,0.565936,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,RandomForestClassifier,Train,0.999978,0.999880,1.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
135,GradientBoostingClassifier,Test,0.872300,0.574400,0.835600,8 SMOTEENN Dataset,0.1277,0.8723,0.5810,0.1674,0.2599,0.9813,0.0187,0.2630
136,AdaBoostClassifier,Train,0.870700,0.582200,0.834800,8 SMOTEENN Dataset,0.1293,0.8707,0.5468,0.1884,0.2802,0.9759,0.0241,0.2668
137,AdaBoostClassifier,Test,0.871500,0.584900,0.832500,8 SMOTEENN Dataset,0.1285,0.8715,0.5580,0.1935,0.2874,0.9763,0.0237,0.2748
138,LogisticRegression,Train,0.869400,0.564800,0.825200,8 SMOTEENN Dataset,0.1306,0.8694,0.5404,0.1493,0.2340,0.9804,0.0196,0.2340


## 4. Filter and sort test results

In [14]:
# Sort Dataframe by column to maximize
maximize = 'Accuracy'
limits = {
    'balanced_accuracy': .5,
    'Precision': .2
}

# Create dataframe with only the test metrics
test_df = df[df['slice'] == 'Test']

limit_df = test_df.copy()

# limit the rows based on the values in the limits dictionary above
for limit in limits:
    print(limit)
    limit_value = limits[limit]
    limit_df = limit_df[limit_df[limit]>limit_value]

# sort by column_to_maximize fro highest to lowest
sorted_df = limit_df.sort_values(by=column_to_maximize, ascending=False, inplace=False)


sorted_df

balanced_accuracy
Precision


NameError: name 'column_to_maximize' is not defined

In [15]:
sorted_df.columns

NameError: name 'sorted_df' is not defined